<a href="https://colab.research.google.com/github/mohmaed7777/knn-algorithm-HR-Analytics-Job-Change-of-Data-Scientists/blob/main/HR_Analytics_Job_Change_of_Data_Scientists.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Description:-**

A company which is active in Big Data and Data Science wants to hire data scientists among people who successfully pass some courses which conduct by the company. Many people signup for their training. Company wants to know which of these candidates are really wants to work for the company after training or looking for a new employment because it helps to reduce the cost and time as well as the quality of training or planning the courses and categorization of candidates. Information related to demographics, education, experience are in hands from candidates signup and enrollment.

This dataset designed to understand the factors that lead a person to leave current job for HR researches too. By model(s) that uses the current credentials,demographics,experience data you will predict the probability of a candidate to look for a new job or will work for the company, as well as interpreting affected factors on employee decision.

In [2]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
%matplotlib inline 
sns.set()
%config InlineBackend.figure_formats=['retina']
%pylab inline
import sklearn 

Populating the interactive namespace from numpy and matplotlib


In [3]:
path_file = '/content/aug_train.csv'
df = pd.read_csv(path_file, sep=',')
df.head()

,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,target
0,8949,city_103,0.920,Male,Has relevent experience,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36,1.0
1,29725,city_40,0.776,Male,No relevent experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47,0.0
2,11561,city_21,0.624,NaN,No relevent experience,Full time course,Graduate,STEM,5,NaN,NaN,never,83,0.0
3,33241,city_115,0.789,NaN,No relevent experience,NaN,Graduate,Business Degree,<1,NaN,Pvt Ltd,never,52,1.0
4,666,city_162,0.767,Male,Has relevent experience,no_enrollment,Masters,STEM,>20,50-99,Funded Startup,4,8,0.0


In [4]:
df.dtypes

enrollee_id                 int64
city                       object
city_development_index    float64
gender                     object
relevent_experience        object
enrolled_university        object
education_level            object
major_discipline           object
experience                 object
company_size               object
company_type               object
last_new_job               object
training_hours              int64
target                    float64
dtype: object

In [5]:
missing_data = df.isnull().sum()
missing_data[:]

enrollee_id                  0
city                         0
city_development_index       0
gender                    4508
relevent_experience          0
enrolled_university        386
education_level            460
major_discipline          2813
experience                  65
company_size              5938
company_type              6140
last_new_job               423
training_hours               0
target                       0
dtype: int64

In [6]:
df.dropna(axis=0, inplace=True)

In [7]:
df.head()

,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,target
1,29725,city_40,0.776,Male,No relevent experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47,0.0
4,666,city_162,0.767,Male,Has relevent experience,no_enrollment,Masters,STEM,>20,50-99,Funded Startup,4,8,0.0
7,402,city_46,0.762,Male,Has relevent experience,no_enrollment,Graduate,STEM,13,<10,Pvt Ltd,>4,18,1.0
8,27107,city_103,0.920,Male,Has relevent experience,no_enrollment,Graduate,STEM,7,50-99,Pvt Ltd,1,46,1.0
11,23853,city_103,0.920,Male,Has relevent experience,no_enrollment,Graduate,STEM,5,5000-9999,Pvt Ltd,1,108,0.0


In [17]:
df.tail()

,gender,relevent_experience,enrolled_university,education_level,major_discipline,training_hours,target
19147,Male,No relevent experience,Full time course,Graduate,STEM,52,1.0
19149,Male,Has relevent experience,no_enrollment,Masters,STEM,36,1.0
19150,Female,Has relevent experience,no_enrollment,Graduate,STEM,23,0.0
19152,Female,Has relevent experience,no_enrollment,Graduate,Humanities,25,0.0
19155,Male,Has relevent experience,no_enrollment,Graduate,STEM,44,0.0


In [8]:
df.drop(['enrollee_id','city','city_development_index','experience','company_size','last_new_job','company_type'], axis=1, inplace=True)

In [9]:
df.head()

,gender,relevent_experience,enrolled_university,education_level,major_discipline,training_hours,target
1,Male,No relevent experience,no_enrollment,Graduate,STEM,47,0.0
4,Male,Has relevent experience,no_enrollment,Masters,STEM,8,0.0
7,Male,Has relevent experience,no_enrollment,Graduate,STEM,18,1.0
8,Male,Has relevent experience,no_enrollment,Graduate,STEM,46,1.0
11,Male,Has relevent experience,no_enrollment,Graduate,STEM,108,0.0


In [10]:
df_uniques = df.nunique()
df_uniques

gender                   3
relevent_experience      2
enrolled_university      3
education_level          3
major_discipline         6
training_hours         241
target                   2
dtype: int64

In [25]:
# Check the Variables that only have Two Values : 
binary_variables = ['relevent_experience']

In [26]:
binary_variables

['relevent_experience']

In [27]:
categorical_variables = list(df_uniques[(df_uniques > 2) & (df_uniques <= 6)].index)
categorical_variables

['gender', 'enrolled_university', 'education_level', 'major_discipline']

In [28]:
[[i, list(df[i].unique())]for i in categorical_variables]

[['gender', ['Male', 'Female', 'Other']],
 ['enrolled_university',
  ['no_enrollment', 'Part time course', 'Full time course']],
 ['education_level', ['Graduate', 'Masters', 'Phd']],
 ['major_discipline',
  ['STEM', 'Humanities', 'Business Degree', 'Other', 'No Major', 'Arts']]]

In [29]:
ordinal_variables = ['gender','enrolled_university','education_level']


**Encoding process:**

In [33]:
from sklearn.preprocessing import LabelBinarizer, LabelEncoder, OrdinalEncoder

In [34]:
lb, le, lo = LabelBinarizer(), LabelEncoder(), OrdinalEncoder()

In [36]:
for column in ordinal_variables:
  df[column] = le.fit_transform(df[column])

In [39]:
for column in binary_variables:
  df[column] = lb.fit_transform(df[column])

In [40]:
categorical_variables = list(set(categorical_variables) - set(ordinal_variables) - set(binary_variables))
categorical_variables

['major_discipline']

In [41]:
df = pd.get_dummies(df, columns= categorical_variables, drop_first=True)

In [42]:
df.head()

,gender,relevent_experience,enrolled_university,education_level,training_hours,target,major_discipline_Business Degree,major_discipline_Humanities,major_discipline_No Major,major_discipline_Other,major_discipline_STEM
1,1,1,2,0,47,0.0,0,0,0,0,1
4,1,0,2,1,8,0.0,0,0,0,0,1
7,1,0,2,0,18,1.0,0,0,0,0,1
8,1,0,2,0,46,1.0,0,0,0,0,1
11,1,0,2,0,108,0.0,0,0,0,0,1


In [43]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
gender,8955.0,0.918928,0.303200,0.0,1.0,1.0,1.0,2.0
relevent_experience,8955.0,0.123283,0.328780,0.0,0.0,0.0,0.0,1.0
enrolled_university,8955.0,1.755109,0.608916,0.0,2.0,2.0,2.0,2.0
education_level,8955.0,0.330207,0.527190,0.0,0.0,0.0,1.0,2.0
training_hours,8955.0,65.074930,60.235087,1.0,23.0,47.0,88.0,336.0
target,8955.0,0.165606,0.371747,0.0,0.0,0.0,0.0,1.0
major_discipline_Business Degree,8955.0,0.018984,0.136475,0.0,0.0,0.0,0.0,1.0
major_discipline_Humanities,8955.0,0.042211,0.201082,0.0,0.0,0.0,0.0,1.0
major_discipline_No Major,8955.0,0.012507,0.111139,0.0,0.0,0.0,0.0,1.0
major_discipline_Other,8955.0,0.019765,0.139201,0.0,0.0,0.0,0.0,1.0


In [44]:
from sklearn.preprocessing import MinMaxScaler
mm = MinMaxScaler()

In [45]:
for column in [ordinal_variables + numaric_variables]:
  df[column] = mm.fit_transform(df[column])

In [46]:
round(df.describe().T, 3)

,count,mean,std,min,25%,50%,75%,max
gender,8955.0,0.459,0.152,0.0,0.500,0.500,0.50,1.0
relevent_experience,8955.0,0.123,0.329,0.0,0.000,0.000,0.00,1.0
enrolled_university,8955.0,0.878,0.304,0.0,1.000,1.000,1.00,1.0
education_level,8955.0,0.165,0.264,0.0,0.000,0.000,0.50,1.0
training_hours,8955.0,0.191,0.180,0.0,0.066,0.137,0.26,1.0
target,8955.0,0.166,0.372,0.0,0.000,0.000,0.00,1.0
major_discipline_Business Degree,8955.0,0.019,0.136,0.0,0.000,0.000,0.00,1.0
major_discipline_Humanities,8955.0,0.042,0.201,0.0,0.000,0.000,0.00,1.0
major_discipline_No Major,8955.0,0.013,0.111,0.0,0.000,0.000,0.00,1.0
major_discipline_Other,8955.0,0.020,0.139,0.0,0.000,0.000,0.00,1.0


**Train Test Split:**

In [52]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score, accuracy_score, classification_report, confusion_matrix

In [53]:
x = df.drop(columns='target')

In [54]:
x.head()

,gender,relevent_experience,enrolled_university,education_level,training_hours,major_discipline_Business Degree,major_discipline_Humanities,major_discipline_No Major,major_discipline_Other,major_discipline_STEM
1,0.5,1,1.0,0.0,0.137313,0,0,0,0,1
4,0.5,0,1.0,0.5,0.020896,0,0,0,0,1
7,0.5,0,1.0,0.0,0.050746,0,0,0,0,1
8,0.5,0,1.0,0.0,0.134328,0,0,0,0,1
11,0.5,0,1.0,0.0,0.319403,0,0,0,0,1


In [56]:
y = df[['target']]
y.head()

,target
1,0.0
4,0.0
7,1.0
8,1.0
11,0.0


In [61]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.4, random_state=42)

In [77]:
# Estimate KNN model and report outcomes : 
knn = KNeighborsClassifier(n_neighbors=3)
knn = knn.fit(x_train, y_train)
y_pred = knn.predict(x_test)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


In [78]:
# precision , recall, f1_score from the multiclass function : 
print(classification_report(y_test, y_pred))
print('accuracy score:', round(accuracy_score(y_test, y_pred),2))
print('f1_score:', round(f1_score(y_test, y_pred), 2))

              precision    recall  f1-score   support

         0.0       0.84      0.92      0.87      2981
         1.0       0.20      0.10      0.14       601

    accuracy                           0.78      3582
   macro avg       0.52      0.51      0.51      3582
weighted avg       0.73      0.78      0.75      3582

accuracy score: 0.78
f1_score: 0.14
